In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import warnings
import datetime

#indications = {0: 'Borderline Personality Disorder', 1: 'Major Depressive Disorder', 2: 'ADHD', 3: 'Bipolar disorder' , 4: 'Substance abuse' , 5: 'PTSD', 6: 'Adjustment disorder' , 7: 'Generalised Anxiety Disorder' , 8: 'Conduct Disorder' , 9: 'Panic Disorder' , 10: 'Schizophrenia' , 11: 'Tourette', 12: 'Resistant Depression', 13: 'Autism', 14: 'Alzheimer'}
indications = pd.read_csv('./Trial.csv')
#print(indications)

def indication_search(indication, options={}, verbose=False):
    """
    Given the name of an indication, search the EU Clinical Trials Register and save the results as a DataFrame
    """
    if verbose:
        print(indication)
        if 'page' in options:
            print(f'Page: {options["page"]}')
        else:
            print('No page specified')
    # Create search URL
    print(indication)

    indication_string = indication.replace(' ', '+')
    search_url = f'https://www.clinicaltrialsregister.eu/ctr-search/search?query={indication_string}'
    
    # retrieve results from the database as an HTML page
    with warnings.catch_warnings(record=True) as cw:
        results_page = requests.get(search_url, verify=False, params=options)
    final_url = results_page.url
    if verbose:
        print(final_url)
    
    # parse the HTML structure using Beautiful Soup
    soup = BeautifulSoup(results_page.text)
    
    # find the number of hits
    hit_count_string = soup.find('a', id='ui-id-1').text.strip()
    n_hits = int(re.search('\(([0-9]+)\)', hit_count_string).group(1))
    # print(n_hits)
    
    if n_hits > 0:
        # get number of hits, pages, and current page
        pages_string = soup.find('div', 'outcome grid_12').text.strip()
        sub_strings = pages_string.split()

        hits = int(sub_strings[0])
        current_page = int(sub_strings[-3])
        n_pages = int(sub_strings[-1].strip('.'))
        if verbose:
            print(f'{hits} hits; page {current_page}/{n_pages}')

        # check that we have found the correct values
        assert hits == n_hits

        if 'page' in options:
            assert current_page == int(options['page'])

        # find all tables of class 'result' - one per trial found
        tables = soup.find_all('table', 'result')

        table_list = []

        # loop through results
        for current_table in tables:
            # find table cells
            data_cells = current_table.find_all('td')
            cell_dict = {}
            # loop through the cells
            for cell in data_cells:
                # does the cell contain a 'label' <span> tag?
                label = cell.find('span', 'label')
                if label:
                    cell_text = cell.text
                    # split cell text on first colon
                    parts = cell.text.split(':', maxsplit=1)
                    # clean up label and value strings
                    label_str = parts[0].strip()
                    value_str = parts[1].strip()
                    # save in a dictionary
                    cell_dict[label_str] = value_str
            # save the dictonary for each cell in a list
            table_list.append(cell_dict)
        # convert the list of dictionaries into a DataFrame
        hit_frame = pd.DataFrame(table_list)

        if current_page < n_pages:  # need to get subsequent page(s)
            new_options = dict(options)
            new_options['page'] = f'{current_page + 1}'
            sub_frame = indication_search(indication, options=new_options)
            combined_frame = hit_frame.append(sub_frame, ignore_index=True)
        else:
            combined_frame = hit_frame

        if 'Disease' in combined_frame.columns:
            combined_frame.drop('Disease', axis=1, inplace=True)

        if current_page == 1:
            rows, cols = combined_frame.shape
            print(f'{indication}: {rows} of {n_hits} records retrieved from {n_pages} pages')
            assert rows == n_hits
    else:
        print(f'{indication}: {n_hits} found')
        combined_frame = None
    return combined_frame


def result_link(in_frame):
  results_available = in_frame['Trial results'].str.match('View results')
  res_link = 'https://www.clinicaltrialsregister.eu/ctr-search/trial/' + in_frame["EudraCT Number"][results_available] + '/results'
  in_frame.loc[results_available, 'Trial results'] = res_link

def protocol_link(row_input):
  """
  Create protocol links for each country in each trial
  """
  stem = 'https://www.clinicaltrialsregister.eu/ctr-search/trial/'
  links = []
  for ccode in row_input.iloc[0:-2]:
      if type(ccode) is str:
          links.append(f'{stem}{row_input.iloc[-1]}/{ccode}')
      else:
          links.append('')
  return pd.Series(links, dtype=str)

def country_links(in_frame):
  countries = in_frame['Trial protocol'].str.extractall(r'([A-Z]{2})\n')
  countries = countries.unstack(level=-1)
  countries.columns = countries.columns.droplevel(0)
  new = countries.join(in_frame["EudraCT Number"])
  link_frame = new.apply(protocol_link, axis=1)
  return link_frame

earliest_start = '2020-01-01'
phase_names = {1: 'one', 2: 'two', 3: 'three', 4: 'four'}

search_options = {'dateFrom': earliest_start, 'phase': ['phase-' + name for name in phase_names.values()]}
option_frame = pd.DataFrame.from_dict(search_options)

frame_list = []
for indication in indications.iterrows():
    ind_frame = indication_search(indication[1]['Indication'], options=search_options)
    if ind_frame is not None:
        result_link(ind_frame)
        link_frame = country_links(ind_frame)
        new_frame = ind_frame.join(link_frame)    
        frame_list.append(new_frame)
    else:
        frame_list.append(None)

merged = pd.concat(frame_list, axis=0, keys=indications['Indication'])
merged.head()

merged.shape

grouped = merged.groupby(['Sponsor Name', 'Indication'])

summary = grouped.count()
all_columns = list(summary.columns)
summary.drop(all_columns[1:], axis=1, inplace=True)
summary.rename(columns={'EudraCT Number': 'Count'})
summary

indication_counts = merged.groupby('Indication').count()['EudraCT Number']
indication_counts

sponsor_counts = merged.groupby('Sponsor Name').count()['EudraCT Number']
sorted_counts = sponsor_counts.sort_values(ascending=False)
pd.DataFrame(sorted_counts).head(14)


now = datetime.datetime.utcnow()
stamp = now.strftime('%Y%m%d_%H%M%S')


xlname = f'EudraCTsearch_{stamp}.xlsx'
print(xlname)
with pd.ExcelWriter(xlname) as xlwriter:
    option_frame.to_excel(xlwriter, sheet_name='Search options')
    merged.to_excel(xlwriter, sheet_name='Merged')
    summary.to_excel(xlwriter, sheet_name='Summary')
    indication_counts.to_excel(xlwriter, sheet_name='Indications')
    sorted_counts.to_excel(xlwriter, sheet_name='Sponsors')
    for ind_series, frame in zip(indications.iterrows(), frame_list):
        if frame is not None:
            frame.to_excel(xlwriter, sheet_name=ind_series[1]['Indication'])


Borderline Personality Disorder
Borderline Personality Disorder: 3 of 3 records retrieved from 1 pages
Major Depressive Disorder
Major Depressive Disorder
Major Depressive Disorder
Major Depressive Disorder: 52 of 52 records retrieved from 3 pages
ADHD
ADHD
ADHD: 22 of 22 records retrieved from 2 pages
Bipolar disorder
Bipolar disorder
Bipolar disorder: 25 of 25 records retrieved from 2 pages
Substance abuse
Substance abuse: 8 of 8 records retrieved from 1 pages
PTSD
PTSD: 13 of 13 records retrieved from 1 pages
Adjustment disorder
Adjustment disorder: 18 of 18 records retrieved from 1 pages
Generalised Anxiety Disorder
Generalised Anxiety Disorder: 1 of 1 records retrieved from 1 pages
Conduct Disorder
Conduct Disorder
Conduct Disorder: 25 of 25 records retrieved from 2 pages
Panic Disorder
Panic Disorder: 0 found
Schizophrenia
Schizophrenia
Schizophrenia
Schizophrenia: 51 of 51 records retrieved from 3 pages
EudraCTsearch_20220826_201426.xlsx


# New section